In [1]:
import pandas as pd
from gurobipy import *
xlsx = pd.ExcelFile('OR_final資料.xlsx')

# print(xlsx.sheet_names)

truck_sheet = pd.read_excel(xlsx, '消防車可服務火災種類')
# print(truck_sheet)

population_sheet = pd.read_excel(xlsx, '各區人口數')
# print(population_sheet)

fire_sheet = pd.read_excel(xlsx, '107-112火災各種類件數')
# print(fire_sheet)

distance_sheet = pd.read_excel(xlsx, '消防站和火災距離是否小於3公里')
# print(distance_sheet)


泡沫消防車->0
乾粉消防車->1
水箱消防車->2

In [2]:
truck_type = truck_sheet.values.tolist()
cost = []
truck_support = []
for row in truck_type:
    # print(row)
    truck_support.append([row[1],row[2],row[3]])
    cost.append(row[4])
print(truck_support)
print(cost)

truck_type_num = len(truck_support)
print(truck_type_num)

[[1, 1, 0], [0, 1, 1], [1, 0, 0]]
[900, 1200, 700]
3


松山區->1
信義區->2
大安區->3
中山區->4
中正區->5
大同區->6
萬華區->7
文山區->8
南港區->9
內湖區->10
士林區->11
北投區->12


In [3]:
population_info = population_sheet.values.tolist()
population = []
for row in population_info:
    population.append(row[1])
print(population)

district_num = len(population)
print(district_num)

[193653, 206356, 291949, 216125, 150208, 119869, 172944, 260023, 113682, 275843, 266410, 242050]
12


普通火災->0	油類火災->1	電氣火災->2


In [4]:
fire_info = fire_sheet.values.tolist()
fire_type = []
for row in fire_info:
    fire_type.append([row[1], row[2], row[3]])
print(fire_type)

fire_type_num = len(fire_type[0])
print(fire_type_num)

[[176.0, 385.0, 247.0], [227.0, 414.0, 231.0], [237.0, 442.0, 366.0], [358.0, 472.0, 414.0], [156.0, 254.0, 269.0], [154.0, 196.0, 176.0], [281.0, 480.0, 307.0], [279.0, 520.0, 230.0], [119.0, 193.0, 120.0], [266.0, 475.0, 262.0], [369.0, 374.0, 344.0], [352.0, 337.0, 227.0]]
3


In [5]:
import numpy as np
distance_sheet = distance_sheet.drop(columns=['1/0'])
covers = distance_sheet.values.tolist()
covers = np.array(covers)
print(covers)

station_num = len(covers)
print(station_num)

[[0 0 1 1 1 1 1 0 0 0 0 0]
 [1 0 1 1 1 1 1 0 0 0 0 0]
 [0 0 0 0 1 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 1 1 1 0 0 0 0 0]
 [0 0 0 0 1 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 1 0 1 0 1 0 0 0 0 0]
 [1 1 1 0 0 0 0 0 0 0 0 0]
 [1 1 1 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0 0 1 0 0 0]
 [1 1 1 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0]
 [1 0 1 1 1 1 0 0 0 0 0 0]
 [1 0 0 1 0 1 0 0 0 0 0 0]
 [1 0 0 1 0 0 0 0 0 0 0 0]
 [1 0 0 1 0 1 0 0 0 0 1 0]
 [1 1 1 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 1 1 0 0]
 [0 0 0 1 1 1 1 0 0 0 0 0]
 [0 0 0 1 1 1 1 0 0 0 0 0]
 [0 0 0 1 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 

Gurobi part:     i->district, j->station, k->fire truck type, r->fire type

In [6]:
eg1 = Model("eg1")

t = []
for j in range(station_num):
    t.append([])
    for k in range(truck_type_num):
        t[j].append(eg1.addVar(lb = 0, vtype = GRB.INTEGER, name = "t" + str(j+1) + str(k+1)))

eg1.update()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-11


In [7]:
eg1.setObjective(quicksum(cost[k] * quicksum(t[j][k] for j in range(station_num)) for k in range(truck_type_num)), GRB.MINIMIZE)

P = 100000
for i in range(district_num):
    eg1.addConstr(P * quicksum(covers[j][i] * quicksum(t[j][k] for k in range(truck_type_num)) for j in range(station_num)) >= population[i])
H = 50
for i in range(district_num):
    for r in range(fire_type_num):
        eg1.addConstr(H * quicksum(covers[j][i] * quicksum(truck_support[k][r] * t[j][k] for k in range(truck_type_num)) for j in range(station_num)) >= fire_type[i][r])



eg1.optimize()  

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1260P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 48 rows, 138 columns and 776 nonzeros
Model fingerprint: 0x778f683a
Variable types: 0 continuous, 138 integer (0 binary)
Coefficient statistics:
  Matrix range     [5e+01, 1e+05]
  Objective range  [7e+02, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 3e+05]
Found heuristic solution: objective 79300.000000
Presolve removed 48 rows and 138 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 2: 61300 79300 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.130000000000e+04, best bound 6.130000000000e+04, gap 0.0000%


In [8]:
if eg1.status == GRB.OPTIMAL:
    total_trucks = 0
    minCost = eg1.objVal
    print(minCost)
else:
    print("No optimal solution found. Model status:", eg1.status)

61300.0


In [9]:
# Prepare data for the DataFrame
data = []
total_trucks = 0
columns = ["Station"] + [f"Truck {k+1}" for k in range(truck_type_num)]

for j in range(station_num):
    row = [f"Station {j+1}"]
    for k in range(truck_type_num):
        truck_count = t[j][k].X
        row.append(truck_count)
        total_trucks += truck_count
    data.append(row)

# Create a DataFrame
df1 = pd.DataFrame(data, columns=columns)
print(df1)

print("Total Truck", total_trucks)

       Station  Truck 1  Truck 2  Truck 3
0    Station 1     -0.0     -0.0     -0.0
1    Station 2      3.0      7.0      3.0
2    Station 3     -0.0     -0.0     -0.0
3    Station 4     -0.0     -0.0     -0.0
4    Station 5     -0.0     -0.0     -0.0
5    Station 6     -0.0     -0.0     -0.0
6    Station 7     -0.0     -0.0     -0.0
7    Station 8      6.0     -0.0     -0.0
8    Station 9     -0.0     -0.0     -0.0
9   Station 10     -0.0     -0.0     -0.0
10  Station 11     -0.0      5.0     -0.0
11  Station 12     -0.0     -0.0     -0.0
12  Station 13     -0.0     -0.0     -0.0
13  Station 14     -0.0     -0.0     -0.0
14  Station 15     -0.0     -0.0     -0.0
15  Station 16     -0.0     -0.0     -0.0
16  Station 17     -0.0     -0.0     -0.0
17  Station 18     -0.0     -0.0     -0.0
18  Station 19     -0.0     -0.0     -0.0
19  Station 20     -0.0     -0.0     -0.0
20  Station 21     -0.0     -0.0     -0.0
21  Station 22     -0.0     -0.0     -0.0
22  Station 23     -0.0     -0.0  

In [10]:
eg2 = Model("eg2")

t = []
for j in range(station_num):
    t.append([])
    for k in range(truck_type_num):
        t[j].append(eg2.addVar(lb = 0, vtype = GRB.INTEGER, name = "t" + str(j+1) + str(k+1)))

w = []
for j in range(station_num):
    w.append(eg2.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "w" + str(j+1)))
    
e = eg2.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "e")

eg2.update()

In [11]:
eg2.setObjective(e, GRB.MINIMIZE)

for j in range(station_num):
    eg2.addConstr(e >= quicksum(cost[k] * t[j][k] for k in range(truck_type_num)) - minCost / station_num)
    eg2.addConstr(w[j] == quicksum(cost[k] * t[j][k] for k in range(truck_type_num)) - minCost / station_num)

#for j in range(station_num):
    #eg2.addConstr(quicksum(t[j][k] for k in range(truck_type_num)) <= 5)
    
for i in range(district_num):
    eg2.addConstr(P * quicksum(covers[j][i] * quicksum(t[j][k] for k in range(truck_type_num)) for j in range(station_num)) >= population[i])

for i in range(district_num):
    for r in range(fire_type_num):
        eg2.addConstr(H * quicksum(covers[j][i] * quicksum(truck_support[k][r] * t[j][k] for k in range(truck_type_num)) for j in range(station_num)) >= fire_type[i][r])



eg2.optimize()  

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1260P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 140 rows, 185 columns and 1144 nonzeros
Model fingerprint: 0x3e633717
Variable types: 47 continuous, 138 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 3e+05]
Found heuristic solution: objective 12667.391304
Presolve removed 14 rows and 61 columns
Presolve time: 0.00s
Presolved: 126 rows, 124 columns, 975 nonzeros
Variable types: 1 continuous, 123 integer (0 binary)

Root relaxation: objective 2.667391e+03, 80 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

 

In [12]:
if eg2.status == GRB.OPTIMAL:
    max_gap = eg2.objVal
    print("Maximum gap:", max_gap)
    print("Budget per station:", minCost / station_num)
else:
    print("No optimal solution found. Model status:", eg2.status)

Maximum gap: 2667.391304347826
Budget per station: 1332.608695652174


In [13]:
# Prepare data for the DataFrame
data = []
total_trucks = 0
columns = ["Station"] + [f"Truck {k+1}" for k in range(truck_type_num)] + ["Exceeding Expense"]

for j in range(station_num):
    row = [f"Station {j+1}"]
    for k in range(truck_type_num):
        truck_count = t[j][k].X
        row.append(truck_count)
        total_trucks += truck_count
    row.append(w[j].X)
    data.append(row)

# Create a DataFrame
df2 = pd.DataFrame(data, columns=columns)
print(df2)
# Write the DataFrame to an Excel file
excel_file = "firetruck_dispatcher_reg.xlsx"
df2.to_excel(excel_file, index=False)

print(f"Table saved to {excel_file}")
print("Maximum gap:", max_gap)
print("Budget per station:", minCost / station_num)
print("Total Truck", total_trucks)


       Station  Truck 1  Truck 2  Truck 3  Exceeding Expense
0    Station 1      0.0      0.0      2.0          67.391304
1    Station 2      0.0      0.0      2.0          67.391304
2    Station 3      0.0      2.0      0.0        1067.391304
3    Station 4      0.0      3.0      0.0        2267.391304
4    Station 5      0.0      2.0      0.0        1067.391304
5    Station 6      2.0      0.0      0.0         467.391304
6    Station 7      0.0      2.0      0.0        1067.391304
7    Station 8      4.0      0.0      0.0        2267.391304
8    Station 9      2.0      0.0      0.0         467.391304
9   Station 10      0.0      3.0      0.0        2267.391304
10  Station 11      0.0      2.0      0.0        1067.391304
11  Station 12      0.0      0.0      2.0          67.391304
12  Station 13      0.0      0.0      2.0          67.391304
13  Station 14      1.0      2.0      0.0        1967.391304
14  Station 15      0.0      0.0      2.0          67.391304
15  Station 16      2.0 